## Algorithm: 

### The end goal is to find a matrix which gives the minimum square loss w.r.t. the original matrix (i.e. reviews). 


$$min_{Z \in \mathbb{R}^{m\times n}}\ \  f(Z) :=  \frac{1}{2}\sum_{(i,j)\in \Omega} \left( Z_{ij} - X_{ij}\right)^{2}$$

Subject to having a low rank (or a small nuclear norm). 


NOTE: It would seem to me that the gradient of this function would simply be: 
$$ \nabla f(Z) = \sum_{ij}(Z_{ij} - X_{ij}) $$
right?


### This found matrix needs to have low rank for two reasons: 
1. It is a constraint that allows for the problem to be well posed. (Otherwise the matrix elements could be anything). 
2. Rank identifies the number of linearly indipendent columns, since we want to build a recommender system we would like there to be relations among the columns, the constraint is not a weakness, it is a desired property.




### How is this matrix found? (i think)

1. The Frank-Wolfe algorithm starts at the zeroth (k) iteration with a guess of a good solution, this solution is called $Z^{k}$. 
2. This solution (it is a matrix) is then used to solve a "sub"-minimization problem. Frank-Wolfe looks for a matrix $Z$ (with nuclear norm below a certain quantity) which minimizes the following quantity: $$ \nabla f(Z^{k}) \cdot Z $$ The matrix which satisfies this is called $\tilde{Z}^{k}$. 
3. Then the algorithm chooses $Z^{k+1}$ by means of a standard stepping procedure: $$ Z^{k+1} = (1-\alpha_{k}) Z^{k} + \alpha_{k}\tilde{Z}^{k}$$ 


Note that the FW algorithms already gives a recipe to calculate the matrix $\tilde{Z}^{k}$ at step 2. It holds that: 
$$ \tilde{Z}^{k} = -\delta u_{1}(v_{1})^{T}$$ is a rank-one matrix. Where $u_{1}, v_{1}$ are the singular vectors (why only 2 ?) associated with the largest singular value of $\nabla f(Z^{k})$.

In [39]:
#------------IDEAS TO APPLY TO ACTUAL DATA---------------

# df.fillna(0)
# function to replace NaNs with 0s. (nobody gives 0 rating, come on.)

# Then I can use https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html
# to search for the singular values of the gradient of the matrix. 

# Then I update the matrix with the procedure in the paper/slides

# Repeat until tolerance/max iterations reached (?)

In [80]:
# (probably won't need this function, maybe as a check on the nuclear norm of the solution at each step (?) ) #



#This function calculates the nuclear norm of a matrix.
# (the nuclear norm is the sum of all non-zero singular values of a matrix)


#the [1] accesses the eigenvalues of the diagonal matrix D -----> X^{T}*D*X
def nuclear_norm(A):
    return np.sum(np.linalg.svd(A)[1])

X = np.array([[1,0,0], [0,1,0], [0, 0, 1]])
Y= np.array([[1,0,1], [0,1,0], [1, 0, 1]])

print('\n Example of nuclear norm:')
print('\n The matrix is: \n', X)
print('\n Its nuclear norm is: ', nuclear_norm(X))


 Example of nuclear norm:

 The matrix is: 
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

 Its nuclear norm is:  3.0


In [51]:
def step(data, previous, delta):
    # might not be a good idea to pass the original data matrix everytime
    
    # ----------calculate gradient here (is it a number? an array? a matrix?)
    
    #-----------find SVD of gradient
    
    #-----------take u_1 and v_1 and delta and create update TILDE ZK (check that it has rank one)
    
    #------------create new iterate Z^{k+1}

    #------------repeat

    

0.0
